# Loading libaries and data

## For Milestone 1 (should also be loaded if attempting to run Milestone 2)

In [1]:
!pip install -q avro fastavro faker geopy pycountry requests beautifulsoup4 pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 27.6 MB/s eta 0:00:00


In [2]:
import csv
import io
import random
import time
from datetime import datetime, timedelta

import avro.schema
import fastavro
import pandas as pd
import pycountry
import pytz
import requests
from avro.datafile import DataFileWriter
from avro.io import DatumWriter
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from faker import Faker
from fastavro import reader, writer, parse_schema

## For Milestone 2

In [3]:
!pip install -q kafka-python findspark pyspark azure-eventhub azure-storage-blob azure-eventhub-checkpointstoreblob azure-storage-queue

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.9 MB/s eta 0:00:00


In [4]:
import os
import sys

import findspark
import pyspark
from azure.storage.queue import QueueClient, QueueServiceClient
from IPython.display import display, clear_output
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import avg, col, expr, from_unixtime, hour, sum, window

## Loading data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_path = '/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Spotify-2000.csv'

df = pd.read_csv(data_path)
df.head(10)

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59
5,6,The Road Ahead (Miles Of The Unknown),City To City,alternative pop rock,2004,99,46,54,-9,14,14,247,0,2,45
6,7,She Will Be Loved,Maroon 5,pop,2002,102,71,71,-6,13,54,257,6,3,74
7,8,Knights of Cydonia,Muse,modern rock,2006,137,96,37,-5,12,21,366,0,14,69
8,9,Mr. Brightside,The Killers,modern rock,2004,148,92,36,-4,10,23,223,0,8,77
9,10,Without Me,Eminem,detroit hip hop,2002,112,67,91,-3,24,66,290,0,7,82


# From Milestone 1

## Creating the schema

In [7]:
# set a timezone-aware start time using UTC timezone
utc = pytz.utc
start_time = datetime(2020, 1, 1, tzinfo=utc)

# AVRO Schema

spotify_wrap_schema = {

  "doc": "User listening history with song details.", # documentation/description of schema

  "name": "ListeningHistory", # name of record type the schema represents

  "namespace": "music", # helps to prevent name clashes with other schema

  "type": "record", # type of schema - record representing a complex structure

  # fields of the recod
  "fields": [
    {"name": "user_id", "type": "int"},

    {
      "name": "user_location",
      "type": {
        "type": "record",
        "name": "Location",
        "fields": [
          {"name": "latitude", "type": "float"},
          {"name": "longitude", "type": "float"},
          {"name": "country", "type": "string"},
          {"name": "city", "type": "string"},
          {"name": "timezone", "type": "string"}
        ]
      }
    },


    {"name": "interaction", "type": "string"},

    {"name": "song_name", "type": "string"},

    {"name": "artist", "type": "string"},

    {"name": "genre", "type": "string"},

    {"name": "year", "type": "int"},

    {"name": "popularity", "type": "int"},

    {"name": "danceability", "type": "float"},

    {"name": "energy", "type": "float"},

    {"name": "length_seconds", "type": "int"},

    {
      "name": "listening_time",
      "type": {
        "type": "record",
        "name": "ListeningPeriod",
        "fields": [
            {"name": "start_time", "type": "long"},
            {"name": "end_time", "type": "long"}
        ]
    }
    },

    {
      "name": "personality",
      "type": "string",
    }
  ]
}

## Initialization

In [8]:
personalities = {
    "The early adopter": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Variety", "C/U": "Commonality"},
    "The nomad": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The specialist": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The enthusiast": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The connoisseur": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The deep diver": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The fanclubber": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Variety", "C/U": "Commonality"},
    "The top charter": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The replayer": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The jukeboxer": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Commonality"},
    "The voyager": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The devotee": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The maverick": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The time traveler": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The musicologist": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Commonality"},
    "The adventurer": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Variety", "C/U": "Uniqueness"}
}

song_df = df[['Title', 'Artist', 'Top Genre', 'Year', 'Popularity', 'Length (Duration)']]
fake = Faker()
user_song_history = {}
records = []
user_country_mapping = {}

## Functions

In [9]:
def dynamic_song_selection(user_id, personality, history, song_df):
    year_preference = random.choice([1990, 2000, 2010, 2020])
    filtered_df = song_df[song_df['Year'] == year_preference]

    if user_id in history:
        filtered_df = filtered_df[~filtered_df['Title'].isin(history[user_id])]

    if filtered_df.empty:
        filtered_df = song_df

    return filtered_df.sample(n=1)

In [10]:
def simulate_interaction():
    interactions = ['played', 'paused', 'skipped', 'repeated', 'added_to_playlist']
    probabilities = [0.5, 0.2, 0.15, 0.1, 0.05]
    return random.choices(interactions, weights=probabilities, k=1)[0]

In [11]:
#This is same as dynamic song selection but takes into account personality
def dynamic_song_selection_2(user_id, personality, user_song_history, df):
    characteristics = personalities[personality]
    feature = random.choice(list(characteristics.keys()))
    value = characteristics[feature]

    if user_id not in user_song_history or not user_song_history[user_id]:
        # If the user has no song history, select a random song based on artist
        random_song = df[df['Artist'].isin(df['Artist'].unique())].sample(n=1)
        user_song_history[user_id] = [random_song]

        return random_song

    # User has song history, apply filtering logic based on personality
    if feature == "F/E":
        filter_condition = df['Artist'].isin(user_song_history[user_id][0]['Artist']) if value == "Familiarity" else ~df['Artist'].isin(user_song_history[user_id][0]['Artist'])
    elif feature == "T/N":
        filter_condition = df['Year'] == user_song_history[user_id][0]['Year'].values[0] if value == "Newness" else ~(df['Year'] == user_song_history[user_id][0]['Year'].values[0])
    elif feature == "L/V":
        filter_condition = df['Title'].isin(user_song_history[user_id][0]['Title']) if value == "Loyalty" else ~df['Title'].isin(user_song_history[user_id][0]['Title'])
    elif feature == "C/U":
        filter_condition = df['Popularity'] > user_song_history[user_id][0]['Popularity'].values[0] if value == "Commonality" else df['Popularity'] < user_song_history[user_id][0]['Popularity'].values[0]

    filtered_df = df[filter_condition].sample(n=1)
    user_song_history[user_id].append(filtered_df)

    # Return the same structure as dynamic_song_selection
    return filtered_df.sample(n=1)

In [12]:
def parse_duration(duration_str):
    # Remove commas and any other non-numeric characters
    duration_str = ''.join(c for c in duration_str if c.isdigit() or c == ':')

    # Split the string into minutes and seconds
    parts = duration_str.split(':')

    if len(parts) == 2:
        minutes, seconds = map(int, parts)
    elif len(parts) == 1:
        minutes = 0
        seconds = int(parts[0])
    else:
        raise ValueError("Invalid duration format: " + duration_str)

    return minutes * 60 + seconds

In [13]:
def generate_valid_country_code():
    while True:
        country_code = fake.country_code(representation="alpha-2")
        # Ensure the country code is compatible with local_latlng
        if fake.local_latlng(country_code=country_code):
            return country_code

In [14]:
def generate_user_location(user_id,user_country_mapping):
    if user_id in user_country_mapping:
        # If the user already has a country assigned, use that country
        random_country = user_country_mapping[user_id]
    else:
        # Generate a new country for the user
        random_country = generate_valid_country_code()
        user_country_mapping[user_id] = random_country

    location = fake.local_latlng(country_code=random_country)
    latitude, longitude, city, country, timezone = location

    return {
        "user_id": user_id,
        "latitude": latitude,
        "longitude": longitude,
        "country": country,
        "city": city,
        "timezone": timezone
    }

In [15]:
def filter_songs(df, feature, value, user_id, history):
    if feature == "F/E":
        exploration_threshold = 0.8
        is_exploring = random.random() > exploration_threshold if value == "Exploration" else random.random() < exploration_threshold
        artist_in_history = df['Artist'].isin(history[user_id]) if user_id in history else pd.Series([False] * len(df))
        return df[artist_in_history] if is_exploring else df[~artist_in_history]
    elif feature == "T/N":
        newness_threshold = 0.8
        is_new = random.random() < newness_threshold if value == "Newness" else random.random() > newness_threshold
        return df[df['Year'] == 2019] if is_new else df[df['Year'] != 2019]
    elif feature == "L/V":
        loyalty_threshold = 0.8
        is_loyal = random.random() < loyalty_threshold if value == "Loyalty" else random.random() > loyalty_threshold
        title_in_history = df['Title'].isin(history[user_id]) if user_id in history else pd.Series([False] * len(df))
        return df[title_in_history] if is_loyal else df[~title_in_history]
    elif feature == "C/U":
        commonality_threshold = 0.8
        is_common = random.random() < commonality_threshold if value == "Commonality" else random.random() > commonality_threshold
        return df[df['Popularity'] > 70] if is_common else df[df['Popularity'] <= 70]
    return df

In [16]:
def generate_record(user_id, personality, df, user_location, start_time, n_records, n_records2):

    user_song_history.setdefault(user_id, [])
    characteristics = personalities[personality]
    feature, value = random.choice(list(characteristics.items()))

    filtered_df = filter_songs(df, feature, value, user_id, user_song_history)

    if filtered_df.empty:
        filtered_df = df

    song_row = filtered_df.sample(n=1).iloc[0]
    user_song_history[user_id].append(song_row['Title'])

    # Time adjustments
    if start_time is not None:
        n_records += 1
        n_records2 += 1
        if n_records > 20:
            start_time += timedelta(days=1)
            n_records = 0
        if n_records2 > 400:
            start_time += relativedelta(months=1)
            n_records2 = 0

    # Interaction handling
    interaction = random.choices(['played', 'paused', 'shuffle'], weights=[0.7, 0.2, 0.1], k=1)[0]
    interaction_duration = random.randint(30, 210) if interaction != 'paused' else random.randint(30, 90)
    end_time = start_time + timedelta(seconds=interaction_duration)

    # Format time consistently
    start_time_ms = int(start_time.timestamp() * 1000)
    end_time_ms = int(end_time.timestamp() * 1000)

    # Return structured record
    return {
        "user_id": user_id,
        "user_location": user_location,
        "interaction": interaction,
        "song_name": song_row['Title'],
        "artist": song_row['Artist'],
        "genre": song_row['Top Genre'],
        "year": int(song_row['Year']),
        "popularity": int(song_row['Popularity']),
        "danceability": float(song_row.get('Danceability', 0)),
        "energy": float(song_row.get('Energy', 0)),
        "length_seconds": int(song_row['Length (Duration)']),
        "listening_time": (start_time_ms, end_time_ms),
        "personality": personality
    }

In [17]:
def simulate_user_session(user_id, start_time, duration_hours, personality, user_song_history, song_df, user_country_mapping):
    session_end_time = start_time + timedelta(hours=duration_hours)
    current_time = start_time
    session_records = []

    #This generates one location per user, so no more changing location after one song (no more teleporting)
    user_location = generate_user_location(user_id, user_country_mapping)

    while current_time < session_end_time:
        # Adjust start time for each song
        song_start_time = current_time

        # Choose a song
        song_row = dynamic_song_selection(user_id, personality, user_song_history, song_df).iloc[0]

        # Simulate interaction
        interaction = simulate_interaction()

        # Determine listening duration based on interaction type
        if interaction == 'skipped':
            listen_duration = random.randint(5, 30)
        elif interaction == 'repeated':
            listen_duration = random.randint(360, 600)
        else:
            listen_duration = random.randint(180, 300)

        # Calculate end time
        end_time = song_start_time + timedelta(seconds=listen_duration)

        # Convert time to milliseconds
        start_time_ms = int(song_start_time.timestamp() * 1000)
        end_time_ms = int(end_time.timestamp() * 1000)

        # Create record
        record = {
            "user_id": user_id,
            "user_location": user_location,
            "interaction": interaction,
            "song_name": song_row['Title'],
            "artist": song_row['Artist'],
            "genre": song_row['Top Genre'],
            "year": song_row['Year'],
            "popularity": song_row['Popularity'],
            "danceability": song_row.get('Danceability', 0),
            "energy": song_row.get('Energy', 0),
            "length_seconds": parse_duration(song_row['Length (Duration)']),
            "listening_time": {"start_time": start_time_ms, "end_time": end_time_ms},
            "personality": personality
        }

        # Append record to session records
        session_records.append(record)

        # Update user song history
        if user_id not in user_song_history:
            user_song_history[user_id] = []
        user_song_history[user_id].append(song_row['Title'])

        # Update current time for next iteration
        current_time = end_time

    return session_records

In [18]:
def main(num_sessions_per_user,session_duration_hours_range,n_users):
  for user_id in range(1,n_users):  # Example user IDs from 1 to 1000
    personality = random.choice(list(personalities.keys()))
    user_sessions_records = []
    for _ in range(num_sessions_per_user):
        personality = random.choice(list(personalities.keys()))
        start_time = fake.date_time_between(start_date="-1y", end_date="now", tzinfo=utc)
        session_duration_hours = random.randint(*session_duration_hours_range)
        # Ensure simulate_user_session or your equivalent function makes use of the new generate_record function
        session_records = simulate_user_session(user_id, start_time, session_duration_hours, personality, user_song_history, song_df, user_country_mapping)
        user_sessions_records.extend(session_records)
    records.extend(user_sessions_records)

  # Serialize data
  with io.BytesIO() as bytes_writer:
      fastavro.writer(bytes_writer, spotify_wrap_schema, records)
      serialized_data = bytes_writer.getvalue()

  # Output the serialized data length
  print("Serialized data length:", len(serialized_data))

  # If you want to save the serialized data to a file
  with open('spotify_wrapped_data.avro', 'wb') as avro_file:
      avro_file.write(serialized_data)

## Running the code

In [19]:
#RUN THIS TO DO ALL IN ONE GO
main(num_sessions_per_user=3,session_duration_hours_range=(1,3),n_users=1000)

Serialized data length: 12175720


In [20]:
num_sessions_per_user = 3
session_duration_hours_range = (1, 3)

for user_id in range(1, 5):  # Example user IDs from 1 to 1000
    personality = random.choice(list(personalities.keys()))
    user_sessions_records = []
    for _ in range(num_sessions_per_user):
        start_time = fake.date_time_between(start_date="-1y", end_date="now", tzinfo=utc)
        session_duration_hours = random.randint(*session_duration_hours_range)
        # Ensure simulate_user_session or your equivalent function makes use of the new generate_record function
        session_records = simulate_user_session(user_id, start_time, session_duration_hours, personality, user_song_history, song_df, user_country_mapping)
        user_sessions_records.extend(session_records)
    records.extend(user_sessions_records)

In [21]:
# Serialize data
with io.BytesIO() as bytes_writer:
    fastavro.writer(bytes_writer, spotify_wrap_schema, records)
    serialized_data = bytes_writer.getvalue()

# Output the serialized data length
print("Serialized data length:", len(serialized_data))

# If you want to save the serialized data to a file
with open('spotify_wrapped_data.avro', 'wb') as avro_file:
    avro_file.write(serialized_data)

Serialized data length: 12225831


In [22]:
# Path to your Avro file
avro_file_path = 'spotify_wrapped_data.avro'

# Open the Avro file
with open(avro_file_path, 'rb') as avro_file:
    # Create a reader object
    reader = fastavro.reader(avro_file)

    # Access the schema of your Avro file (optional)
    schema = reader.schema
    print("Schema:", schema)

    # Iterate through the records in the Avro file
    #for record in reader:
        #print(record)


Schema: {'doc': 'User listening history with song details.', 'name': 'ListeningHistory', 'namespace': 'music', 'type': 'record', 'fields': [{'name': 'user_id', 'type': 'int'}, {'name': 'user_location', 'type': {'type': 'record', 'name': 'Location', 'fields': [{'name': 'latitude', 'type': 'float'}, {'name': 'longitude', 'type': 'float'}, {'name': 'country', 'type': 'string'}, {'name': 'city', 'type': 'string'}, {'name': 'timezone', 'type': 'string'}]}}, {'name': 'interaction', 'type': 'string'}, {'name': 'song_name', 'type': 'string'}, {'name': 'artist', 'type': 'string'}, {'name': 'genre', 'type': 'string'}, {'name': 'year', 'type': 'int'}, {'name': 'popularity', 'type': 'int'}, {'name': 'danceability', 'type': 'float'}, {'name': 'energy', 'type': 'float'}, {'name': 'length_seconds', 'type': 'int'}, {'name': 'listening_time', 'type': {'type': 'record', 'name': 'ListeningPeriod', 'fields': [{'name': 'start_time', 'type': 'long'}, {'name': 'end_time', 'type': 'long'}]}}, {'name': 'person

<ipython-input-22-a794202f8186>:10: DeprecationWarning: The 'schema' attribute is deprecated. Please use 'writer_schema'
  schema = reader.schema


# Milestone 2

## Part 1: Kafka

### Kafka Environment Set-up

In [23]:
%%writefile environment.sh
#!/usr/bin/bash
export KAFKA_BINARY_VERSION='3.7.0'
export SCALA_BINARY_VERSION='2.13'
export KAFKA_BINARY_VERSION=$KAFKA_BINARY_VERSION
export SCALA_BINARY_VERSION=$SCALA_BINARY_VERSION
export PATH=$PATH:$PWD/kafka_$SCALA_BINARY_VERSION-$KAFKA_BINARY_VERSION/bin

Writing environment.sh


In [24]:
%%writefile kafka_setup.sh

# Source environment variables
source ./environment.sh

# Print Kafka and Scala versions
echo "Kafka Version: kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}"
echo "Current PATH: $PATH"

# Java Setup
wget -O- https://apt.corretto.aws/corretto.key | sudo apt-key add -
sudo add-apt-repository 'deb https://apt.corretto.aws stable main' -y
sudo apt-get -y update; sudo apt-get install -y java-11-amazon-corretto-jdk

# Kafka Setup
wget https://downloads.apache.org/kafka/${KAFKA_BINARY_VERSION}/kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}.tgz
tar xzf kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}.tgz

UUID=$(./kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}/bin/kafka-storage.sh random-uuid)
echo "export UUID=$UUID" >> ./environment.sh
cat environment.sh

# Start Kafka Broker

echo kafka_$SCALA_BINARY_VERSION-$KAFKA_BINARY_VERSION

# offsets.retention.minutes determines how long Kafka retains the commit offsets for consumer groups.
echo "offsets.retention.minutes=300" >> ./kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}/config/kraft/server.properties

./kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}/bin/kafka-storage.sh format -t ${UUID} -c ./kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}/config/kraft/server.properties
nohup ./kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}/bin/kafka-server-start.sh ./kafka_${SCALA_BINARY_VERSION}-${KAFKA_BINARY_VERSION}/config/kraft/server.properties > kafka_server.log &

Writing kafka_setup.sh


In [25]:
%%shell
source kafka_setup.sh
sleep 10
tail -20 kafka_server.log

Kafka Version: kafka_2.13-3.7.0
Current PATH: /opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/content/kafka_2.13-3.7.0/bin
--2024-04-13 19:20:42--  https://apt.corretto.aws/corretto.key
Resolving apt.corretto.aws (apt.corretto.aws)... 18.66.255.23, 18.66.255.118, 18.66.255.24, ...
Connecting to apt.corretto.aws (apt.corretto.aws)|18.66.255.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1695 (1.7K) [binary/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.66K  --.-KB/s    in 0s      

2024-04-13 19:20:43 (80.2 MB/s) - written to stdout [1695/1695]

OK
Repository: 'deb https://apt.corretto.aws stable main'
Description:
Archive for codename: stable components: main
More info: https://apt.corretto.aws
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/archive_uri-https_apt_corretto_aws-jammy.list
Adding di

Create a Kafka environment with partitions = 16, given that we have 16 personalities, and we want to analyze by personality.

In [26]:
%%shell

source ./environment.sh

kafka-topics.sh --bootstrap-server 127.0.0.1:9092 --topic spotifyWrapGroup4 --create --partitions 16 --replication-factor 1

Created topic spotifyWrapGroup4.


Let's look at the Consumer Group

In [27]:
%%writefile check_kafka_consumers.sh
#!/usr/bin/env bash
source ./environment.sh

echo "Active Consumer Groups"
while true
do
date
kafka-consumer-groups.sh --bootstrap-server 127.0.0.1:9092 --describe --all-groups
sleep 1
done

Writing check_kafka_consumers.sh


In [28]:
%%shell
chmod +x ./check_kafka_consumers.sh
nohup ./check_kafka_consumers.sh > kafka_consumers.log &

nohup: redirecting stderr to stdout


### Kafka Producer of Spotify AVRO Records

In [29]:
kafka_brokers="127.0.0.1:9092"
topic_name="spotifyWrapGroup4"

In [30]:
%%writefile avro_producer.py
#!/usr/bin/python

# Import the required modules
from kafka import KafkaProducer
import fastavro
import io
import random
from faker import Faker
from datetime import datetime, timedelta
import pytz
import pycountry
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

utc = pytz.utc
start_time = datetime(2020, 1, 1, tzinfo=utc)

# schema:
spotify_wrap_schema = {'doc': 'User listening history with song details.',
 'name': 'ListeningHistory',
 'namespace': 'music',
 'type': 'record',
 'fields': [{'name': 'user_id', 'type': 'int'},
  {'name': 'user_location',
   'type': {'type': 'record',
    'name': 'Location',
    'fields': [{'name': 'latitude', 'type': 'float'},
     {'name': 'longitude', 'type': 'float'},
     {'name': 'country', 'type': 'string'},
     {'name': 'city', 'type': 'string'},
     {'name': 'timezone', 'type': 'string'}]}},
  {'name': 'interaction', 'type': 'string'},
  {'name': 'song_name', 'type': 'string'},
  {'name': 'artist', 'type': 'string'},
  {'name': 'genre', 'type': 'string'},
  {'name': 'year', 'type': 'int'},
  {'name': 'popularity', 'type': 'int'},
  {'name': 'danceability', 'type': 'float'},
  {'name': 'energy', 'type': 'float'},
  {'name': 'length_seconds', 'type': 'int'},
  {'name': 'listening_time',
   'type': {'type': 'record',
    'name': 'ListeningPeriod',
    'fields': [{'name': 'start_time', 'type': 'long'},
     {'name': 'end_time', 'type': 'long'}]}},
  {'name': 'personality', 'type': 'string'}]}

folder_path = '/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/'

df = pd.read_csv(folder_path + 'Spotify-2000.csv')

personalities = {
    "The early adopter": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Variety", "C/U": "Commonality"},
    "The nomad": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The specialist": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The enthusiast": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The connoisseur": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The deep diver": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The fanclubber": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Variety", "C/U": "Commonality"},
    "The top charter": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The replayer": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The jukeboxer": {"F/E": "Familiarity", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Commonality"},
    "The voyager": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Commonality"},
    "The devotee": {"F/E": "Familiarity", "T/N": "Newness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The maverick": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Loyalty", "C/U": "Uniqueness"},
    "The time traveler": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Uniqueness"},
    "The musicologist": {"F/E": "Exploration", "T/N": "Timelessness", "L/V": "Variety", "C/U": "Commonality"},
    "The adventurer": {"F/E": "Exploration", "T/N": "Newness", "L/V": "Variety", "C/U": "Uniqueness"}
}

song_df = df[['Title', 'Artist', 'Top Genre', 'Year', 'Popularity', 'Length (Duration)']]
fake = Faker()
user_song_history = {}
records = []
user_country_mapping = {}

def dynamic_song_selection(user_id, personality, history, song_df):
    year_preference = random.choice([1990, 2000, 2010, 2020])
    filtered_df = song_df[song_df['Year'] == year_preference]

    if user_id in history:
        filtered_df = filtered_df[~filtered_df['Title'].isin(history[user_id])]

    if filtered_df.empty:
        filtered_df = song_df

    return filtered_df.sample(n=1)

def simulate_interaction():
    interactions = ['played', 'paused', 'skipped', 'repeated', 'added_to_playlist']
    probabilities = [0.5, 0.2, 0.15, 0.1, 0.05]
    return random.choices(interactions, weights=probabilities, k=1)[0]

def parse_duration(duration_str):
    # Remove commas and any other non-numeric characters
    duration_str = ''.join(c for c in duration_str if c.isdigit() or c == ':')

    # Split the string into minutes and seconds
    parts = duration_str.split(':')

    if len(parts) == 2:
        minutes, seconds = map(int, parts)
    elif len(parts) == 1:
        minutes = 0
        seconds = int(parts[0])
    else:
        raise ValueError("Invalid duration format: " + duration_str)

    return minutes * 60 + seconds

def generate_valid_country_code():
    while True:
        country_code = fake.country_code(representation="alpha-2")
        # Ensure the country code is compatible with local_latlng
        location = fake.local_latlng(country_code=country_code)
        if location is not None:
            return country_code

def generate_user_location(user_id,user_country_mapping):
    if user_id in user_country_mapping:
        # If the user already has a country assigned, use that country
        random_country = user_country_mapping[user_id]
    else:
        # Generate a new country for the user
        random_country = generate_valid_country_code()
        user_country_mapping[user_id] = random_country

    location = fake.local_latlng(country_code=random_country)
    latitude, longitude, city, country, timezone = location

    return {
        "user_id": user_id,
        "latitude": latitude,
        "longitude": longitude,
        "country": country,
        "city": city,
        "timezone": timezone
    }

def filter_songs(df, feature, value, user_id, history):
    if feature == "F/E":
        exploration_threshold = 0.8
        is_exploring = random.random() > exploration_threshold if value == "Exploration" else random.random() < exploration_threshold
        artist_in_history = df['Artist'].isin(history[user_id]) if user_id in history else pd.Series([False] * len(df))
        return df[artist_in_history] if is_exploring else df[~artist_in_history]
    elif feature == "T/N":
        newness_threshold = 0.8
        is_new = random.random() < newness_threshold if value == "Newness" else random.random() > newness_threshold
        return df[df['Year'] == 2019] if is_new else df[df['Year'] != 2019]
    elif feature == "L/V":
        loyalty_threshold = 0.8
        is_loyal = random.random() < loyalty_threshold if value == "Loyalty" else random.random() > loyalty_threshold
        title_in_history = df['Title'].isin(history[user_id]) if user_id in history else pd.Series([False] * len(df))
        return df[title_in_history] if is_loyal else df[~title_in_history]
    elif feature == "C/U":
        commonality_threshold = 0.8
        is_common = random.random() < commonality_threshold if value == "Commonality" else random.random() > commonality_threshold
        return df[df['Popularity'] > 70] if is_common else df[df['Popularity'] <= 70]
    return df


def generate_record(user_id, personality, df, user_location, start_time, n_records, n_records2):

    user_song_history.setdefault(user_id, [])
    characteristics = personalities[personality]
    feature, value = random.choice(list(characteristics.items()))

    filtered_df = filter_songs(df, feature, value, user_id, user_song_history)

    if filtered_df.empty:
        filtered_df = df

    song_row = filtered_df.sample(n=1).iloc[0]
    user_song_history[user_id].append(song_row['Title'])

    # Time adjustments
    if start_time is not None:
        n_records += 1
        n_records2 += 1
        if n_records > 20:
            start_time += timedelta(days=1)
            n_records = 0
        if n_records2 > 400:
            start_time += relativedelta(months=1)
            n_records2 = 0

    # Interaction handling
    interaction = random.choices(['played', 'paused', 'shuffle'], weights=[0.7, 0.2, 0.1], k=1)[0]
    interaction_duration = random.randint(30, 210) if interaction != 'paused' else random.randint(30, 90)
    end_time = start_time + timedelta(seconds=interaction_duration)

    # Format time consistently
    start_time_ms = int(start_time.timestamp() * 1000)
    end_time_ms = int(end_time.timestamp() * 1000)

    # Return structured record
    return {
        "user_id": user_id,
        "user_location": user_location,
        "interaction": interaction,
        "song_name": song_row['Title'],
        "artist": song_row['Artist'],
        "genre": song_row['Top Genre'],
        "year": int(song_row['Year']),
        "popularity": int(song_row['Popularity']),
        "danceability": float(song_row.get('Danceability', 0)),
        "energy": float(song_row.get('Energy', 0)),
        "length_seconds": int(song_row['Length (Duration)']),
        "listening_time": {"start_time": start_time_ms, "end_time": end_time_ms},
        "personality": personality
    }

def simulate_user_session(user_id, start_time, duration_hours, personality, user_song_history, song_df, user_country_mapping):
    session_end_time = start_time + timedelta(hours=duration_hours)
    current_time = start_time
    session_records = []

    #This generates one location per user, so no more changing location after one song (no more teleporting)
    user_location = generate_user_location(user_id, user_country_mapping)

    while current_time < session_end_time:
        # Adjust start time for each song
        song_start_time = current_time

        # Choose a song
        song_row = dynamic_song_selection(user_id, personality, user_song_history, song_df).iloc[0]

        # Simulate interaction
        interaction = simulate_interaction()

        # Determine listening duration based on interaction type
        if interaction == 'skipped':
            listen_duration = random.randint(5, 30)
        elif interaction == 'repeated':
            listen_duration = random.randint(360, 600)
        else:
            listen_duration = random.randint(180, 300)

        # Calculate end time
        end_time = song_start_time + timedelta(seconds=listen_duration)

        # Convert time to milliseconds
        start_time_ms = int(song_start_time.timestamp() * 1000)
        end_time_ms = int(end_time.timestamp() * 1000)



        # Create record
        record = {
            "user_id": user_id,
            "user_location": user_location,
            "interaction": interaction,
            "song_name": song_row['Title'],
            "artist": song_row['Artist'],
            "genre": song_row['Top Genre'],
            "year": song_row['Year'],
            "popularity": song_row['Popularity'],
            "danceability": song_row.get('Danceability', 0),
            "energy": song_row.get('Energy', 0),
            "length_seconds": parse_duration(song_row['Length (Duration)']),
            "listening_time": {"start_time": start_time_ms, "end_time": end_time_ms},
            "personality": personality
        }

        # Append record to session records
        session_records.append(record)

        # Update user song history
        if user_id not in user_song_history:
            user_song_history[user_id] = []
        user_song_history[user_id].append(song_row['Title'])

        # Update current time for next iteration
        current_time = end_time

    return session_records

# Parse the updated schema
from fastavro import parse_schema
parsed_updated_schema = parse_schema(spotify_wrap_schema)

# Create a Kafka producer with value serializer
def serialize(message):
    if isinstance(message, dict):
        print("Serializing message:", message)  # Print out the message for debugging
        schemaless_bytes_writer = io.BytesIO()
        fastavro.schemaless_writer(schemaless_bytes_writer, parsed_updated_schema, message)
        serialized_message = schemaless_bytes_writer.getvalue()
        print("Serialized message:", serialized_message)  # Print out the serialized message for debugging
        return serialized_message
    else:
        return message


producer = KafkaProducer(
    bootstrap_servers=['127.0.0.1:9092'],
    value_serializer=serialize
)


# Generate some records and send them to the topic
# topic name is a command line argument, if not present it defaults to spotifyWrapGroup4
import sys
topic_name_default="spotifyWrapGroup4"
if len(sys.argv) > 1:
  topic_name = sys.argv[1]
else:
  topic_name = topic_name_default

# Function to generate a random record (modified 'main' function)
# Function to generate a random record (modified 'main' function)
def main(num_sessions_per_user, session_duration_hours_range, n_users):
    all_user_sessions_records = []
    for user_id in range(1, n_users + 1):  # Include the last user ID
        personality = random.choice(list(personalities.keys()))
        user_sessions_records = []
        for _ in range(num_sessions_per_user):
            start_time = fake.date_time_between(start_date="-1y", end_date="now", tzinfo=utc)
            session_duration_hours = random.randint(*session_duration_hours_range)
            session_records = simulate_user_session(user_id, start_time, session_duration_hours, personality,
                                                     user_song_history, song_df, user_country_mapping)
            user_sessions_records.extend(session_records)
            all_user_sessions_records.extend(session_records)
    print("All user session records:", all_user_sessions_records)  # Print out all user session records for debugging
    return all_user_sessions_records

messages = main(num_sessions_per_user=3,session_duration_hours_range=(1,3),n_users=10)

# No need to print messages here
for message in messages:
    print("Sending message:", message)  # Print out the message for debugging
    serialized_message = serialize(message)  # Serialize the message
    producer.send(topic_name, value=serialized_message)
    time.sleep(5)

# Flush the producer
producer.flush()

Writing avro_producer.py


In [31]:
%%writefile avro_producer2.py
#!/usr/bin/python

# Import the required modules
from kafka import KafkaProducer
import fastavro
import io
import random
from faker import Faker
from datetime import datetime, timedelta
import pytz
import pycountry
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# schema:
spotify_wrap_schema = {'doc': 'User listening history with song details.',
 'name': 'ListeningHistory',
 'namespace': 'music',
 'type': 'record',
 'fields': [{'name': 'user_id', 'type': 'int'},
  {'name': 'user_location',
   'type': {'type': 'record',
    'name': 'Location',
    'fields': [{'name': 'latitude', 'type': 'float'},
     {'name': 'longitude', 'type': 'float'},
     {'name': 'country', 'type': 'string'},
     {'name': 'city', 'type': 'string'},
     {'name': 'timezone', 'type': 'string'}]}},
  {'name': 'interaction', 'type': 'string'},
  {'name': 'song_name', 'type': 'string'},
  {'name': 'artist', 'type': 'string'},
  {'name': 'genre', 'type': 'string'},
  {'name': 'year', 'type': 'int'},
  {'name': 'popularity', 'type': 'int'},
  {'name': 'danceability', 'type': 'float'},
  {'name': 'energy', 'type': 'float'},
  {'name': 'length_seconds', 'type': 'int'},
  {'name': 'listening_time',
   'type': {'type': 'record',
    'name': 'ListeningPeriod',
    'fields': [{'name': 'start_time', 'type': 'long'},
     {'name': 'end_time', 'type': 'long'}]}},
  {'name': 'personality', 'type': 'string'}]}

def read_avro_records(avro_file_path):
    with open(avro_file_path, 'rb') as avro_file:
        reader = fastavro.reader(avro_file)
        for record in reader:
            yield record

# Kafka Producer with direct byte passing as value_serializer just passes through the bytes
producer = KafkaProducer(
    bootstrap_servers=['127.0.0.1:9092'],
    value_serializer=lambda x: x  # Assuming the Avro records are already serialized
)

# Specify your Avro file path here
avro_file_path = '/content/spotify_wrapped_data.avro'

# Specify the Kafka topic
import sys
topic_name_default="spotifyWrapGroup4"
if len(sys.argv) > 1:
  topic_name = sys.argv[1]
else:
  topic_name = topic_name_default


# Read Avro records and publish to Kafka
for record in read_avro_records(avro_file_path):
    bytes_io = io.BytesIO()  # Create a BytesIO object
    fastavro.schemaless_writer(bytes_io, spotify_wrap_schema, record)  # Write the record to the BytesIO object
    serialized_message = bytes_io.getvalue()  # Get the byte value of the serialized record
    print(record)
    producer.send(topic_name, value=serialized_message)
    time.sleep(0.1)

# Flush and close the producer
producer.flush()
producer.close()

print("Finished sending Avro records to Kafka topic.")

Writing avro_producer2.py


The producer is sent to the background as it will produce records over a long period of time

In [32]:
!nohup python avro_producer.py > avro_producer.log &

nohup: redirecting stderr to stdout


In [33]:
!nohup python avro_producer2.py > avro_producer2.log &

nohup: redirecting stderr to stdout


In [34]:
!sleep 5
!tail -20 avro_producer2.log

In [35]:
!sleep 5
!tail -20 avro_producer.log

All user session records: [{'user_id': 1, 'user_location': {'user_id': 1, 'latitude': '35.89972', 'longitude': '14.51472', 'country': 'MT', 'city': 'Valletta', 'timezone': 'Europe/Malta'}, 'interaction': 'skipped', 'song_name': 'Man On The Moon', 'artist': 'R.E.M.', 'genre': 'alternative rock', 'year': 1992, 'popularity': 67, 'danceability': 0, 'energy': 0, 'length_seconds': 314, 'listening_time': {'start_time': 1691823212220, 'end_time': 1691823226220}, 'personality': 'The voyager'}, {'user_id': 1, 'user_location': {'user_id': 1, 'latitude': '35.89972', 'longitude': '14.51472', 'country': 'MT', 'city': 'Valletta', 'timezone': 'Europe/Malta'}, 'interaction': 'played', 'song_name': 'Peter Gunn Theme', 'artist': 'Emerson, Lake & Palmer', 'genre': 'album rock', 'year': 2010, 'popularity': 14, 'danceability': 0, 'energy': 0, 'length_seconds': 217, 'listening_time': {'start_time': 1691823226220, 'end_time': 1691823492220}, 'personality': 'The voyager'}, {'user_id': 1, 'user_location': {'use

In [36]:
!tail -20 avro_producer.log

All user session records: [{'user_id': 1, 'user_location': {'user_id': 1, 'latitude': '35.89972', 'longitude': '14.51472', 'country': 'MT', 'city': 'Valletta', 'timezone': 'Europe/Malta'}, 'interaction': 'skipped', 'song_name': 'Man On The Moon', 'artist': 'R.E.M.', 'genre': 'alternative rock', 'year': 1992, 'popularity': 67, 'danceability': 0, 'energy': 0, 'length_seconds': 314, 'listening_time': {'start_time': 1691823212220, 'end_time': 1691823226220}, 'personality': 'The voyager'}, {'user_id': 1, 'user_location': {'user_id': 1, 'latitude': '35.89972', 'longitude': '14.51472', 'country': 'MT', 'city': 'Valletta', 'timezone': 'Europe/Malta'}, 'interaction': 'played', 'song_name': 'Peter Gunn Theme', 'artist': 'Emerson, Lake & Palmer', 'genre': 'album rock', 'year': 2010, 'popularity': 14, 'danceability': 0, 'energy': 0, 'length_seconds': 217, 'listening_time': {'start_time': 1691823226220, 'end_time': 1691823492220}, 'personality': 'The voyager'}, {'user_id': 1, 'user_location': {'use

Python Consumer Set Up

In [37]:
%%writefile avro_consumer.py

# Import the required modules
from kafka import KafkaConsumer
import fastavro
import io

# Define the schema for the records
spotify_wrap_schema = {'doc': 'User listening history with song details.',
 'name': 'ListeningHistory',
 'namespace': 'music',
 'type': 'record',
 'fields': [{'name': 'user_id', 'type': 'int'},
  {'name': 'user_location',
   'type': {'type': 'record',
    'name': 'Location',
    'fields': [{'name': 'latitude', 'type': 'float'},
     {'name': 'longitude', 'type': 'float'},
     {'name': 'country', 'type': 'string'},
     {'name': 'city', 'type': 'string'},
     {'name': 'timezone', 'type': 'string'}]}},
  {'name': 'interaction', 'type': 'string'},
  {'name': 'song_name', 'type': 'string'},
  {'name': 'artist', 'type': 'string'},
  {'name': 'genre', 'type': 'string'},
  {'name': 'year', 'type': 'int'},
  {'name': 'popularity', 'type': 'int'},
  {'name': 'danceability', 'type': 'float'},
  {'name': 'energy', 'type': 'float'},
  {'name': 'length_seconds', 'type': 'int'},
  {'name': 'listening_time',
   'type': {'type': 'record',
    'name': 'ListeningPeriod',
    'fields': [{'name': 'start_time', 'type': 'long'},
     {'name': 'end_time', 'type': 'long'}]}},
  {'name': 'personality', 'type': 'string'}]}

# Parse the updated schema
from fastavro import parse_schema
parsed_updated_schema = parse_schema(spotify_wrap_schema)

# Create a Kafka producer with value serializer
def deserialize(message):
  # print("Deserialize:" + str(message))

  import io
  schemaless_bytes_reader = io.BytesIO(message)
  try:
    record=fastavro.schemaless_reader(schemaless_bytes_reader, spotify_wrap_schema)
    return record
  except Exception as ex:
    print(ex)
    return "pass"

# Create a Kafka consumer with value deserializer
# topic name is a command line argument, if not present it defaults to spotifyWrapGroup4
import sys
topic_name_default="spotifyWrapGroup4"
if len(sys.argv) > 1:
  topic_name = sys.argv[1]
else:
  topic_name = topic_name_default

consumer = KafkaConsumer(
    topic_name,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='Python_Spotify_AVRO_Consumer',
    value_deserializer=deserialize #lambda v: fastavro.schemaless_reader(io.BytesIO(v), spotify_wrap_schema)
)

# Consume messages from the topic and print them
for message in consumer:
    print("="*10)
    print(message.value)


Writing avro_consumer.py


In [38]:
!nohup python avro_consumer.py > avro_consumer.log &

nohup: redirecting stderr to stdout


In [39]:
!sleep 5
!tail -20 avro_consumer.log

In [40]:
!ps -ef |grep avro

root        7345       1 22 19:21 ?        00:00:03 python3 avro_producer.py
root        7369       1 10 19:21 ?        00:00:01 python3 avro_producer2.py
root        8129       1  4 19:21 ?        00:00:00 python3 avro_consumer.py
root        8544     257  0 19:21 ?        00:00:00 /bin/bash -c ps -ef |grep avro
root        8546    8544  0 19:21 ?        00:00:00 grep avro


In [41]:
%%shell
source ./environment.sh
kafka-consumer-groups.sh --bootstrap-server localhost:9092 --list

Python_Spotify_AVRO_Consumer


In [42]:
%%shell
source ./environment.sh
kafka-consumer-groups.sh --bootstrap-server localhost:9092 --describe --group Python_Spotify_AVRO_Consumer


GROUP                        TOPIC             PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             CONSUMER-ID                                             HOST            CLIENT-ID
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 15         14              15              1               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 4          18              18              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 13         17              19              2               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 8          17              20              3               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Con

In [43]:
!tail -20 kafka_consumers.log

Sat Apr 13 07:21:50 PM UTC 2024

GROUP                        TOPIC             PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             CONSUMER-ID                                             HOST            CLIENT-ID
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 15         14              14              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 4          18              18              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 13         17              17              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 8          17              17              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-pyth

In [44]:
%%shell
source ./environment.sh
kafka-consumer-groups.sh --bootstrap-server localhost:9092 --describe --all-groups


GROUP                        TOPIC             PARTITION  CURRENT-OFFSET  LOG-END-OFFSET  LAG             CONSUMER-ID                                             HOST            CLIENT-ID
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 15         18              19              1               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 4          26              26              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 13         27              27              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Consumer spotifyWrapGroup4 8          24              24              0               kafka-python-2.0.2-cdb73de0-daa4-4407-a928-431713759f93 /127.0.0.1      kafka-python-2.0.2
Python_Spotify_AVRO_Con

## Part 2: Spark Setup

In [45]:
spark_release='spark-3.5.1'
hadoop_version='hadoop3'

start=time.time()
os.environ['SPARK_RELEASE']=spark_release
os.environ['HADOOP_VERSION']=hadoop_version
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_release}-bin-{hadoop_version}"

In [46]:
# Run below commands in google colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # install Java8
!wget -q http://apache.osuosl.org/spark/${SPARK_RELEASE}/${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # download spark-3.3.X
!tar xf ${SPARK_RELEASE}-bin-${HADOOP_VERSION}.tgz # unzip it

!pip install -q findspark # install findspark
# findspark find your Spark Distribution and sets necessary environment variables

import findspark
findspark.init()

# Check the pyspark version
import pyspark
print(pyspark.__version__)

3.5.1


Create your Spark session. You must define details of the Kafka Cluster to connect to, topic name and consumer group name.
kafka_brokers: List of Kafka bootstrap servers
topic_name: The Kafka topic to read messages from
consumer_group: This allows you to use different Spark jobs to consume the same topic messages and implement different analytics
schema: the AVRO schema of topic messages

In [47]:
kafka_brokers="127.0.0.1:9092" # Can be a comma-separated list of brokers
topic_name="spotifyWrapGroup4"

In [48]:
spotify_wrap_schema = """
{
  "doc": "User listening history with song details.",
  "name": "ListeningHistory",
  "namespace": "music",
  "type": "record",
  "fields": [
    {"name": "user_id", "type": "int"},
    {
      "name": "user_location",
      "type": {
        "type": "record",
        "name": "Location",
        "fields": [
          {"name": "latitude", "type": "float"},
          {"name": "longitude", "type": "float"},
          {"name": "country", "type": "string"},
          {"name": "city", "type": "string"},
          {"name": "timezone", "type": "string"}
        ]
      }
    },
    {"name": "interaction", "type": "string"},
    {"name": "song_name", "type": "string"},
    {"name": "artist", "type": "string"},
    {"name": "genre", "type": "string"},
    {"name": "year", "type": "int"},
    {"name": "popularity", "type": "int"},
    {"name": "danceability", "type": "float"},
    {"name": "energy", "type": "float"},
    {"name": "length_seconds", "type": "int"},
    {
      "name": "listening_time",
      "type": {
        "type": "record",
        "name": "ListeningPeriod",
        "fields": [
          {"name": "start_time", "type": "long"},
          {"name": "end_time", "type": "long"}
        ]
      }
    },
    {
      "name": "personality",
      "type": "string"
    }
  ]
}
"""

In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro

# Create a Spark session
spark = SparkSession \
    .builder \
    .appName("StreamingAVROFromKafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-avro_2.12:3.5.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()

In [50]:
# Kafka Configuration for reading from Kafka/Event Hub
# Kafka source will create a unique group id for each query automatically. The user can set the prefix of the automatically
# generated group.id’s via the optional source option groupIdPrefix, default value is “spark-kafka-source”.
kafkaConf = {
    "kafka.bootstrap.servers": kafka_brokers,
    # Below settins required if kafka is secured:
    # "kafka.sasl.mechanism": "PLAIN",
    # "kafka.security.protocol": "SASL_SSL",
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://eventhubname.servicebus.windows.net/;SharedAccessKeyName=listenpolicyforspark;SharedAccessKey=ckNkSjcyXKGN8FCIRIS3qtkKvW+AEhB6QPaM=;EntityPath=instructortest";',
    "subscribe": topic_name, # to read from specific partitions use option: "assign": {topic_name:[0,1]})
    "startingOffsets": "latest", # "earliest", "latest"
    "enable.auto.commit": "true ",
    "groupIdPrefix": "Stream_Analytics_",
    "auto.commit.interval.ms": "5000"
}


# Read from Event Hub using Kafka
df = spark \
    .readStream \
    .format("kafka") \
    .options(**kafkaConf)

In [51]:
df = df.load()  # Start reading data from the specified Kafka topic

In [52]:
# Deserialize the AVRO messages from the value column
df = df.select(from_avro(df.value, spotify_wrap_schema).alias("spotifyWrapGroup4"))

# Print the schema of the DataFrame
df.printSchema()

root
 |-- spotifyWrapGroup4: struct (nullable = true)
 |    |-- user_id: integer (nullable = false)
 |    |-- user_location: struct (nullable = false)
 |    |    |-- latitude: float (nullable = false)
 |    |    |-- longitude: float (nullable = false)
 |    |    |-- country: string (nullable = false)
 |    |    |-- city: string (nullable = false)
 |    |    |-- timezone: string (nullable = false)
 |    |-- interaction: string (nullable = false)
 |    |-- song_name: string (nullable = false)
 |    |-- artist: string (nullable = false)
 |    |-- genre: string (nullable = false)
 |    |-- year: integer (nullable = false)
 |    |-- popularity: integer (nullable = false)
 |    |-- danceability: float (nullable = false)
 |    |-- energy: float (nullable = false)
 |    |-- length_seconds: integer (nullable = false)
 |    |-- listening_time: struct (nullable = false)
 |    |    |-- start_time: long (nullable = false)
 |    |    |-- end_time: long (nullable = false)
 |    |-- personality: strin

In [53]:
from pyspark.sql.functions import col

# Now we flatten the AVRO record to put each field in a separate column:
df = df.select(
    col("spotifyWrapGroup4.user_id").alias("user_id"),
    col("spotifyWrapGroup4.user_location.latitude").alias("latitude"),
    col("spotifyWrapGroup4.user_location.longitude").alias("longitude"),
    col("spotifyWrapGroup4.user_location.country").alias("country"),
    col("spotifyWrapGroup4.user_location.city").alias("city"),
    col("spotifyWrapGroup4.user_location.timezone").alias("timezone"),
    col("spotifyWrapGroup4.interaction").alias("interaction"),
    col("spotifyWrapGroup4.song_name").alias("song_name"),
    col("spotifyWrapGroup4.artist").alias("artist"),
    col("spotifyWrapGroup4.genre").alias("genre"),
    col("spotifyWrapGroup4.year").alias("year"),
    col("spotifyWrapGroup4.popularity").alias("popularity"),
    col("spotifyWrapGroup4.danceability").alias("danceability"),
    col("spotifyWrapGroup4.energy").alias("energy"),
    col("spotifyWrapGroup4.length_seconds").alias("length_seconds"),
    col("spotifyWrapGroup4.listening_time.start_time").alias("start_time"),
    col("spotifyWrapGroup4.listening_time.end_time").alias("end_time"),
    col("spotifyWrapGroup4.personality").alias("personality")
)

df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- song_name: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- length_seconds: integer (nullable = true)
 |-- start_time: long (nullable = true)
 |-- end_time: long (nullable = true)
 |-- personality: string (nullable = true)



## Part 3: Setting up Queries, Azure Queues, and Window

In [ ]:
!mkdir checkpoint

In [ ]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- timezone: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- song_name: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- danceability: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- length_seconds: integer (nullable = true)
 |-- start_time: long (nullable = true)
 |-- end_time: long (nullable = true)
 |-- personality: string (nullable = true)



### Example Query

In [ ]:
spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [ ]:
# Filter the records by condition and temperature
nomad_long_df = df.filter((df.personality == "The nomad") & (df.length_seconds > 60))

# Display the filtered records in the console
query_name='nomad_60'
query=nomad_long_df.writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName(query_name) \
    .start()

    # .option("checkpointLocation", "checkpoint") \  # Checkpoint not valid for memory

spark.sql('show tables').show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         | nomad_60|       true|
+---------+---------+-----------+



In [ ]:
# Get the list of active streaming queries
active_queries = spark.streams.active

# Print details about each active query
for query in active_queries:
    print(f"Query Name: {query.name}")
    print(f"Query ID: {query.id}")
    print(f"Query Status: {query.status}")
    print(f"Is Query Active: {query.isActive}")
    print("-" * 50)

Query Name: nomad_60
Query ID: 1b2142f0-b75f-4080-a237-9564468202ee
Query Status: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}
Is Query Active: True
--------------------------------------------------


In [ ]:
numTries=5 # if None, itereate forever
i=0
query_name='nomad_60'

while not numTries or i < numTries:
  i += 1
  query_status = query.status.get('message')
  while query_status == 'Processing new data' or query_status == 'Waiting for data to arrive' or query_status == 'Waiting for next trigger':
    query_status = query.status.get('message')
    # print(query.status)
    # print('.', end='')
    # time.sleep(5)
  # clear_output(wait=True)
  print(query_status)
  print(spark.sql(f'SELECT * FROM {query_name}').show(20, truncate=True))

Getting offsets from KafkaV2[Subscribe[spotifyWrapGroup4]]
+-------+--------+---------+-------+----------+---------------+-----------------+--------------------+-------------------+------------------+----+----------+------------+------+--------------+-------------+-------------+-----------+
|user_id|latitude|longitude|country|      city|       timezone|      interaction|           song_name|             artist|             genre|year|popularity|danceability|energy|length_seconds|   start_time|     end_time|personality|
+-------+--------+---------+-------+----------+---------------+-----------------+--------------------+-------------------+------------------+----+----------+------------+------+--------------+-------------+-------------+-----------+
|     49| 11.8659|  34.3869|     SD|Ar Ruseris|Africa/Khartoum|           paused|      Dansen Aan Zee|               BLØF|         dutch pop|2000|        52|         0.0|   0.0|           275|1703766104029|1703766391029|  The nomad|
|     49|

In [ ]:
for query in spark.streams.active:
    if query.name == "nomad_60":
        query.stop()

### Azure Queues

Based on the personality they have, they will go to the corresponding queue in Azure!


#### Set up

In [ ]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Write to Azure Queues") \
    .getOrCreate()

# Define the connection parameters for Azure Storage Account
storage_account_name = "spotifywrapped"
storage_account_key = "GVEPY1eutTYmo6YA7NpTloEVF+0olJ5CyRoj1XtClyEv2USNwNzmUROAXmMqN0I+q76nlDC0UUdm+AStdjmGzA=="
connection_string="DefaultEndpointsProtocol=https;AccountName=spotifywrapped;AccountKey=GVEPY1eutTYmo6YA7NpTloEVF+0olJ5CyRoj1XtClyEv2USNwNzmUROAXmMqN0I+q76nlDC0UUdm+AStdjmGzA==;EndpointSuffix=core.windows.net".format(
  account_name=storage_account_name,
  account_key=storage_account_key
)

# Define the personality queues
personality_queues = {
    'The early adopter': "queue-1",
    'The nomad': "queue-2",
    'The specialist': "queue-3",
    'The enthusiast': "queue-4",
    'The connoisseur': "queue-5",
    'The deep diver': "queue-6",
    'The fanclubber': "queue-7",
    'The top charter': "queue-8",
    'The replayer': "queue-9",
    'The jukeboxer': "queue-10",
    'The voyager': "queue-11",
    'The devotee': "queue-12",
    'The maverick': "queue-13",
    'The time traveler': "queue-14",
    'The musicologist': "queue-15",
    'The adventurer': "queue-16"
}

In [ ]:
# Send each record to Azure Storage Queue based on personality
def send_to_queue(row):
    personality = row.personality
    if personality in personality_queues:
        queue_name = personality_queues[personality]
        queue_client = QueueClient.from_connection_string(connection_string, queue_name)
        queue_client.send_message(row.asDict())
        print(f"Message sent to queue for personality '{personality}'")
    else:
        print(f"Personality '{personality}' not found in queue_clients dictionary. Message not sent to any queue.")

# Write the records to the Azure Storage Queue
query = df.writeStream \
                  .foreach(send_to_queue) \
                  .start()

# Await termination
query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

This needs to run continuously to send to queues, so if you stop it, it will stop sending records to queues

In [ ]:
query.stop()

### Fixed Windows

In [ ]:
df1 = df

In [ ]:
# Create a SparkSession (assuming it's already created as per your setup)
spark = SparkSession.builder \
    .appName("Top Songs at Night Analysis") \
    .getOrCreate()

# Assuming 'df1' is your DataFrame prepared with 'start_time', 'interaction', etc.

# Convert 'start_time' from epoch milliseconds to a timestamp
df1 = df1.withColumn("timestamp", (col("start_time") / 1000).cast("timestamp"))

# Filter for "played" interactions
played_df = df1.filter(col("interaction") == "played")

# Define window parameters (you might adjust according to your data's timestamp frequency and needs)
windowDuration = "1 hour"  # Window duration
slideDuration = "30 minutes"  # Sliding duration

# Assuming 'song_name' is the column you want to aggregate on
# Group by song_name within each sliding window, counting occurrences
song_counts_window = played_df \
    .groupBy(
        window(col("timestamp"), windowDuration, slideDuration),
        col("song_name")
    ) \
    .count() \
    .orderBy(col("count").desc())

# Note: Directly limiting to the "top 10" in the streaming query is challenging due to the nature of streaming data
# The aggregation will track counts, and you can filter when querying the result

query_name = 'top_songs_at_night'

# Start the streaming query, storing in memory for SQL querying
query = song_counts_window.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Monitor the streaming query and fetch results periodically
try:
    for _ in range(100):  # Adjust the range as necessary for your monitoring needs
        clear_output(wait=True)
        spark.sql(f"SELECT window, song_name, count FROM {query_name} LIMIT 10").show(truncate=False)
        time.sleep(10)  # Adjust sleep time as needed
finally:
    query.stop()  # Ensure the query is stopped when done


+------+---------+-----+
|window|song_name|count|
+------+---------+-----+
+------+---------+-----+

+------------------------------------------+-------------------+-----+
|window                                    |song_name          |count|
+------------------------------------------+-------------------+-----+
|{2023-05-06 12:30:00, 2023-05-06 13:30:00}|Candy              |1    |
|{2023-05-06 12:30:00, 2023-05-06 13:30:00}|Still Got The Blues|1    |
|{2023-05-06 13:00:00, 2023-05-06 14:00:00}|Candy              |1    |
|{2023-05-06 13:00:00, 2023-05-06 14:00:00}|Still Got The Blues|1    |
+------------------------------------------+-------------------+-----+

+------------------------------------------+--------------------------+-----+
|window                                    |song_name                 |count|
+------------------------------------------+--------------------------+-----+
|{2023-04-25 15:00:00, 2023-04-25 16:00:00}|Such a Shame              |1    |
|{2023-10-07 07:30

In [ ]:
spark.sql(f"SELECT window, song_name, count FROM {query_name} LIMIT 10").show(truncate=False)

+------------------------------------------+--------------------------+-----+
|window                                    |song_name                 |count|
+------------------------------------------+--------------------------+-----+
|{2023-05-22 08:30:00, 2023-05-22 09:30:00}|More Than Words           |2    |
|{2024-03-17 01:00:00, 2024-03-17 02:00:00}|Wonderful Days            |2    |
|{2023-08-21 16:30:00, 2023-08-21 17:30:00}|Still Got The Blues       |2    |
|{2023-12-16 23:00:00, 2023-12-17 00:00:00}|Nothing Compares 2 U      |2    |
|{2023-07-01 08:00:00, 2023-07-01 09:00:00}|What It Is                |2    |
|{2023-08-21 16:30:00, 2023-08-21 17:30:00}|Such a Shame              |2    |
|{2023-07-10 09:30:00, 2023-07-10 10:30:00}|Atmosphere - 2010 Remaster|2    |
|{2024-01-16 22:00:00, 2024-01-16 23:00:00}|Mag Ik Dan Bij Jou        |2    |
|{2024-03-02 18:00:00, 2024-03-02 19:00:00}|Wind Of Change            |2    |
|{2023-12-16 22:30:00, 2023-12-16 23:30:00}|Nothing Compares 2 U

### Check Active Queries

In [ ]:
# Get the list of active streaming queries
active_queries = spark.streams.active

# Print details about each active query
for query in active_queries:
    print(f"Query Name: {query.name}")
    print(f"Query ID: {query.id}")
    print(f"Query Status: {query.status}")
    print(f"Is Query Active: {query.isActive}")
    print("-" * 50)


Query Name: countries_per_personality_3
Query ID: 9cef00da-9d87-452f-aeb5-c257be186b0c
Query Status: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}
Is Query Active: True
--------------------------------------------------


## ANALYSIS 1: Country analysis (most listened to artist, most listened to song, etc per country)

#### Top country per personality

In [ ]:
# Selecting personality and country columns
selected_df = df.select('personality', 'country')

# Display the top 10 countries per personality
query_name = 'countries_per_personality'

query = selected_df.writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Now when you execute "show tables", it should show the temporary table
spark.sql('show tables').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|         |countries_per_per...|       true|
|         |countries_per_per...|       true|
|         |countries_per_per...|       true|
|         |            nomad_60|       true|
|         |  top_songs_at_night|       true|
+---------+--------------------+-----------+



In [ ]:
# Wait for the stream to finish
#query.awaitTermination()
query_name = 'countries_per_personality'

# SQL to display the top 10 countries per personality
sql_query_cp3 = """
SELECT personality, country, occurrences
FROM (
    SELECT personality, country, occurrences,
           ROW_NUMBER() OVER(PARTITION BY personality ORDER BY occurrences DESC) as rank
    FROM (
        SELECT personality, country, COUNT(*) AS occurrences
        FROM {}
        GROUP BY personality, country
    ) counted
) ranked
WHERE rank <= 10
""".format(query_name)

# Execute the SQL query
result_cp3 = spark.sql(sql_query_cp3)
result_cp3.show()

+----------------+-------+-----------+
|     personality|country|occurrences|
+----------------+-------+-----------+
| The connoisseur|     DO|          3|
|     The devotee|     IN|         33|
|     The devotee|     RU|         11|
|  The enthusiast|     IN|         52|
|The musicologist|     AM|         16|
| The top charter|     IN|         30|
+----------------+-------+-----------+



In [ ]:
result_cp3.show()

+---------------+-------+-----------+
|    personality|country|occurrences|
+---------------+-------+-----------+
| The adventurer|     VU|        166|
| The adventurer|     AD|        126|
| The adventurer|     CI|        125|
| The adventurer|     ID|        109|
| The adventurer|     MY|        109|
| The adventurer|     PS|        107|
| The adventurer|     IE|         97|
| The adventurer|     FI|         94|
| The adventurer|     BE|         91|
| The adventurer|     RS|         79|
|The connoisseur|     DO|        145|
|The connoisseur|     VE|        133|
|The connoisseur|     HR|        117|
|The connoisseur|     UG|        100|
|The connoisseur|     MT|         89|
|The connoisseur|     MZ|         88|
|The connoisseur|     CI|         87|
|The connoisseur|     PT|         86|
|The connoisseur|     AM|         82|
|The connoisseur|     AR|         82|
+---------------+-------+-----------+
only showing top 20 rows



In [ ]:
result_cp3_pd = result_cp3.toPandas()

In [ ]:
# For further analysis later
result_cp3_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_countryperpersonality.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "countries_per_personality":
        query.stop()

#### Top artist per country

In [ ]:
selected_df = df.select('country', 'artist')

# Display the top artist per country
query_name = 'top_artist_per_country'

query = selected_df.writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# SQL to select the top artist per country
sql_query_artist_country = """
SELECT country, artist, occurrences
FROM (
    SELECT country, artist, COUNT(*) AS occurrences,
           ROW_NUMBER() OVER(PARTITION BY country ORDER BY COUNT(*) DESC) as rank
    FROM {}
    GROUP BY country, artist
) ranked
WHERE rank = 1
""".format(query_name)

# Execute the SQL query
result_artist_country = spark.sql(sql_query_artist_country)
result_artist_country.show()

+-------+------+-----------+
|country|artist|occurrences|
+-------+------+-----------+
+-------+------+-----------+



In [ ]:
# Execute the SQL query
result_artist_country.show()

+-------+-----------+-----------+
|country|     artist|occurrences|
+-------+-----------+-----------+
|     AD|         U2|         16|
|     AF|         U2|         21|
|     AM|The Beatles|         29|
|     AO|The Beatles|         26|
|     AR|The Beatles|         11|
|     AT|The Beatles|         13|
|     AU|   Coldplay|         10|
|     AZ|The Beatles|         11|
|     BA|The Beatles|          9|
|     BD|The Beatles|          3|
|     BE|The Beatles|         25|
|     BG|   Coldplay|         14|
|     BJ|         U2|         27|
|     BO|         U2|         16|
|     BR|     Racoon|          8|
|     BW|         U2|         25|
|     BY|The Beatles|          9|
|     CA|         U2|         23|
|     CD|         U2|         12|
|     CF|The Beatles|         19|
+-------+-----------+-----------+
only showing top 20 rows



In [ ]:
result_artist_country_pd = result_artist_country.toPandas()

In [ ]:
result_artist_country_pd.shape[0]

129

In [ ]:
result_artist_country_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_artistcountry.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "top_artist_per_country":
        query.stop()

## ANALYSIS 2: Day vs Night Listening Behavior


### Night Playlist


In [ ]:
#'start_time' is the column in epoch milliseconds needing conversion to human readable timestamp for analysis

df1 = df.withColumn("start_time", (col("start_time") / 1000).cast("timestamp"))
df1 = df1.withColumn("end_time", (col("end_time") / 1000).cast("timestamp"))

# Now, extracting the hour from the 'start_time' column
df1 = df1.withColumn("start_hour", hour(col("start_time")))
# Optionally, if you also want to extract the hour from 'end_time'
df1 = df1.withColumn("end_hour", hour(col("end_time")))

NIGHT PLAYS - dataframe with songs played at night (from 10 pm to 6 am)

In [ ]:
from pyspark.sql.functions import col, when

night_hours = when((col("start_hour") >= 22) | (col("start_hour") < 6), True).otherwise(False)

# Filter for interactions that are plays and occur at night
night_plays = df1.filter((col("interaction") == "played") & night_hours)

# Group by song_name (and country if desired), count the number of plays
most_played_at_night = night_plays.groupBy("song_name").count().orderBy(col("count").desc())
#most_played_at_night_country = night_plays.groupBy("song_name", "country").count().orderBy(col("count").desc()) -- replace if want by country

# No need to change the streaming query setup unless you want to query this specific DataFrame
query_name = 'most_played_at_night'

# Start the streaming query, storing in memory for SQL querying
query = most_played_at_night.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(query_name) \
    .start()

sql_query_night = f"""
SELECT *
FROM {query_name}
ORDER BY count DESC
LIMIT 10
"""

result_night = spark.sql(sql_query_night)
result_night.show(truncate=False)

+---------+-----+
|song_name|count|
+---------+-----+
+---------+-----+



In [ ]:
result_night.show(truncate=False)

+----------------------------------------------------+-----+
|song_name                                           |count|
+----------------------------------------------------+-----+
|Night Fever - From "Saturday Night Fever" Soundtrack|117  |
|Domino                                              |113  |
|Thunderstruck                                       |112  |
|Met Hart En Ziel                                    |112  |
|Such a Shame                                        |110  |
|Love Rears Its Ugly Head                            |109  |
|Enjoy the Silence - Single Mix                      |106  |
|Nothing Compares 2 U                                |104  |
|Silent Lucidity - Remastered/2003                   |104  |
|Send Me An Angel                                    |104  |
+----------------------------------------------------+-----+



In [ ]:
result_night_pd = result_night.toPandas()

In [ ]:
result_night_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_night.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "most_played_at_night":
        query.stop()

### Day Playlist

In [ ]:
morning_hours = when((col("start_hour") >= 6) & (col("start_hour") < 12), True).otherwise(False)

# Filter for interactions that are plays and occur at night
morning_plays = df1.filter((col("interaction") == "played") & morning_hours)

# Group by song_name (and country if desired), count the number of plays
most_played_in_morning = morning_plays.groupBy("song_name").count().orderBy(col("count").desc())

# No need to change the streaming query setup unless you want to query this specific DataFrame
query_name = 'most_played_in_morning'

# Start the streaming query, storing in memory for SQL querying
query = most_played_in_morning.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(query_name) \
    .start()

sql_query_morning = f"""
SELECT *
FROM {query_name}
ORDER BY count DESC
LIMIT 10
"""

result_morning = spark.sql(sql_query_morning)
result_morning.show(truncate=False)

+---------+-----+
|song_name|count|
+---------+-----+
+---------+-----+



In [ ]:
result_morning.show(truncate=False)

+------------------------------+-----+
|song_name                     |count|
+------------------------------+-----+
|Candy                         |95   |
|Papa                          |88   |
|Nothing Compares 2 U          |87   |
|More Than Words               |85   |
|Don't Go Breaking My Heart    |85   |
|Hunger Strike                 |82   |
|Enjoy the Silence - Single Mix|81   |
|Such a Shame                  |81   |
|Domino                        |80   |
|Send Me An Angel              |80   |
+------------------------------+-----+



In [ ]:
result_morning_pd = result_morning.toPandas()

In [ ]:
result_morning_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_night.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "most_played_in_morning":
        query.stop()

### Day vs Night in same DF

In [ ]:
from pyspark.sql.functions import col, from_unixtime, hour, when
from pyspark.sql import SparkSession

#'start_time' is the column in epoch milliseconds needing conversion to human readable timestamp for analysis

df1 = df.withColumn("start_time", (col("start_time") / 1000).cast("timestamp"))
df1 = df1.withColumn("end_time", (col("end_time") / 1000).cast("timestamp"))

# Now, extracting the hour from the 'start_time' column
df1 = df1.withColumn("start_hour", hour(col("start_time")))
# Optionally, if you also want to extract the hour from 'end_time'
df1 = df1.withColumn("end_hour", hour(col("end_time")))

# Define conditions for morning, afternoon, evening, and night hours
morning_hours = when((col("start_hour") >= 6) & (col("start_hour") < 12), "morning").otherwise("")
afternoon_hours = when((col("start_hour") >= 12) & (col("start_hour") < 18), "afternoon").otherwise("")
evening_hours = when((col("start_hour") >= 18) & (col("start_hour") < 0), "evening").otherwise("")
night_hours = when((col("start_hour") >= 0) | (col("start_hour") < 6), "night").otherwise("")

# Filter for interactions that are plays
plays = df1.filter(col("interaction") == "played")

# Add a new column indicating whether it's morning, afternoon, evening, or night
plays_with_timeofday = plays.withColumn("time_of_day",
                                         when(morning_hours == "morning", "morning")
                                        .otherwise(
                                            when(afternoon_hours == "afternoon", "afternoon")
                                            .otherwise(
                                                when(evening_hours == "evening", "evening")
                                                .otherwise(night_hours)
                                            )
                                        )
                                      )

# Group by song_name and time_of_day, count the number of plays
most_played_time_of_day = plays_with_timeofday.groupBy("song_name", "time_of_day").count().orderBy(col("count").desc())

# No need to change the streaming query setup unless you want to query this specific DataFrame
query_name = 'most_played_time_of_day5'

# Start the streaming query, storing in memory for SQL querying
query = most_played_time_of_day.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Query for the most played songs during each time period
sql_query_time_of_day = f"""
SELECT *
FROM {query_name}
ORDER BY count DESC
LIMIT 10
"""

result_time_of_day = spark.sql(sql_query_time_of_day)
result_time_of_day.show(truncate=False)

+---------+-----------+-----+
|song_name|time_of_day|count|
+---------+-----------+-----+
+---------+-----------+-----+



In [ ]:
result_time_of_day.show(truncate=False)

+----------------------------------------------------+-----------+------------------------------------------+-----+
|song_name                                           |time_of_day|window                                    |count|
+----------------------------------------------------+-----------+------------------------------------------+-----+
|I Want To Hold Your Hand - Remastered 2015          |night      |{2024-01-08 18:00:00, 2024-01-09 00:00:00}|4    |
|Hold On Tight                                       |morning    |{2023-12-26 06:00:00, 2023-12-26 12:00:00}|4    |
|Hunger Strike                                       |night      |{2023-10-31 18:00:00, 2023-11-01 00:00:00}|4    |
|Night Fever - From "Saturday Night Fever" Soundtrack|night      |{2023-04-22 18:00:00, 2023-04-23 00:00:00}|4    |
|Enjoy the Silence - Single Mix                      |morning    |{2023-11-24 06:00:00, 2023-11-24 12:00:00}|4    |
|Hard To Handle                                      |night      |{2023-

In [ ]:
result_time_of_day_pd = result_time_of_day.toPandas()

In [ ]:
result_time_of_day_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_day.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "most_played_time_of_day5":
        query.stop()

### Day vs Night + Windows

In [ ]:
from pyspark.sql.functions import window, col, from_unixtime, hour, when, count
from pyspark.sql import SparkSession

#'start_time' is the column in epoch milliseconds needing conversion to human readable timestamp for analysis

df1 = df.withColumn("start_time", (col("start_time") / 1000).cast("timestamp"))
df1 = df1.withColumn("end_time", (col("end_time") / 1000).cast("timestamp"))

# Now, extracting the hour from the 'start_time' column
df1 = df1.withColumn("start_hour", hour(col("start_time")))
# Optionally, if you also want to extract the hour from 'end_time'
df1 = df1.withColumn("end_hour", hour(col("end_time")))

# Define conditions for morning, afternoon, evening, and night hours
morning_hours = when((col("start_hour") >= 6) & (col("start_hour") < 12), "morning").otherwise("")
afternoon_hours = when((col("start_hour") >= 12) & (col("start_hour") < 18), "afternoon").otherwise("")
evening_hours = when((col("start_hour") >= 18) & (col("start_hour") < 0), "evening").otherwise("")
night_hours = when((col("start_hour") >= 0) | (col("start_hour") < 6), "night").otherwise("")

# Filter for interactions that are plays
plays = df1.filter(col("interaction") == "played")

# Add a new column indicating whether it's morning, afternoon, evening, or night
plays_with_timeofday = plays.withColumn("time_of_day",
                                         when(morning_hours == "morning", "morning")
                                        .otherwise(
                                            when(afternoon_hours == "afternoon", "afternoon")
                                            .otherwise(
                                                when(evening_hours == "evening", "evening")
                                                .otherwise(night_hours)
                                            )
                                        )
                                      )

# Create 6-hour fixed windows with 1-hour watermark
window_duration = "6 hours"
watermark = "1 hour"

# Group by song_name, time_of_day, and window, count the number of plays
most_played_time_of_day = plays_with_timeofday \
    .withWatermark("start_time", watermark) \
    .groupBy("song_name", "time_of_day", window("start_time", window_duration)) \
    .count() \
    .orderBy(col("count").desc())

# No need to change the streaming query setup unless you want to query this specific DataFrame
query_name = 'most_played_time_of_day'

# Start the streaming query, storing in memory for SQL querying
query = most_played_time_of_day.writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Query for the most played songs during each time period
sql_query_time_of_day = f"""
SELECT *
FROM {query_name}
ORDER BY count DESC
LIMIT 10
"""

result_time_of_day = spark.sql(sql_query_time_of_day)
result_time_of_day.show(truncate=False)

+---------+-----------+------+-----+
|song_name|time_of_day|window|count|
+---------+-----------+------+-----+
+---------+-----------+------+-----+



In [ ]:
result_time_of_day.show(truncate=False)

+----------------------------------------------------+-----------+------------------------------------------+-----+
|song_name                                           |time_of_day|window                                    |count|
+----------------------------------------------------+-----------+------------------------------------------+-----+
|I Want To Hold Your Hand - Remastered 2015          |night      |{2024-01-08 18:00:00, 2024-01-09 00:00:00}|4    |
|Hold On Tight                                       |morning    |{2023-12-26 06:00:00, 2023-12-26 12:00:00}|4    |
|Hunger Strike                                       |night      |{2023-10-31 18:00:00, 2023-11-01 00:00:00}|4    |
|Night Fever - From "Saturday Night Fever" Soundtrack|night      |{2023-04-22 18:00:00, 2023-04-23 00:00:00}|4    |
|Enjoy the Silence - Single Mix                      |morning    |{2023-11-24 06:00:00, 2023-11-24 12:00:00}|4    |
|Hard To Handle                                      |night      |{2023-

In [ ]:
result_time_of_day_pd = result_time_of_day.toPandas()

In [ ]:
result_time_of_day_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_timeofday.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "most_played_time_of_day":
        query.stop()

## ANALYSIS 3: Listening stats per Personality

### Most popular Song, Artist and Genre per Personality

In [54]:
query_name = 'df_stats_per_personality'

query = df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Find the most used interaction, song_name, artist, and genre per personality
sql_query_most_used = f"""
SELECT
    personality,
    interaction AS Top_interaction,
    song_name AS most_listened_song,
    artist AS most_listened_artist,
    genre AS most_listened_genre,
    COUNT(*) AS count
FROM {query_name}
GROUP BY personality, interaction, song_name, artist, genre
ORDER BY count DESC
"""

# Execute the queries
result_most_used = spark.sql(sql_query_most_used)

In [57]:
print("Most Used Interaction, Song Name, Artist, and Genre per Personality:")
result_most_used.show(truncate=False)

Most Used Interaction, Song Name, Artist, and Genre per Personality:
+--------------+-----------------+----------------------------------------------------+--------------------+-------------------+-----+
|personality   |Top_interaction  |most_listened_song                                  |most_listened_artist|most_listened_genre|count|
+--------------+-----------------+----------------------------------------------------+--------------------+-------------------+-----+
|The adventurer|played           |Back Down South                                     |Kings of Leon       |modern rock        |1    |
|The fanclubber|played           |Gold                                                |Spandau Ballet      |dance rock         |1    |
|The replayer  |paused           |Als Het Golft                                       |De Dijk             |dutch indie        |1    |
|The voyager   |played           |Paradise                                            |Coldplay            |permanent wav

In [ ]:
result_most_used_pd = result_most_used.toPandas()

In [ ]:
result_most_used_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_mostused.csv', index=False)

### Average listening time per personality

In [56]:
selected_df2 = df.select('user_id', 'personality', 'artist', 'interaction', 'length_seconds')
played_df2 = selected_df2.filter(col('interaction') == 'played')

# Assuming 'length_seconds' column is of string type, cast it to integer
played_df2 = played_df2.withColumn("length_seconds", played_df2["length_seconds"].cast("int"))

# Aggregate data by personality, calculating both total and average listening time
listening_time_df = played_df2.groupBy('personality') \
    .agg(sum('length_seconds').alias('total_listening_time_seconds'),
         avg('length_seconds').alias('average_listening_time_seconds'))

# Convert total listening time to minutes
listening_time_df = listening_time_df.withColumn('total_listening_time_minutes',
                                                 listening_time_df['total_listening_time_seconds'] / 60.0)

# Start the streaming query
query_name = 'listening_time_query'
query = listening_time_df.writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# SQL query to select personality, total listening time, and average listening time
sql_query_listeners = """
    SELECT personality, total_listening_time_minutes, average_listening_time_seconds
    FROM listening_time_query
"""

# Execute the SQL query and display the result
result_listeners = spark.sql(sql_query_listeners)
result_listeners.show(truncate=False)

+-----------+----------------------------+------------------------------+
|personality|total_listening_time_minutes|average_listening_time_seconds|
+-----------+----------------------------+------------------------------+
+-----------+----------------------------+------------------------------+



In [62]:
result_listeners.show(truncate=False)

+-----------------+----------------------------+------------------------------+
|personality      |total_listening_time_minutes|average_listening_time_seconds|
+-----------------+----------------------------+------------------------------+
|The jukeboxer    |3.1333333333333333          |188.0                         |
|The maverick     |89.31666666666666           |255.1904761904762             |
|The jukeboxer    |74.58333333333333           |248.61111111111111            |
|The maverick     |139.05                      |238.37142857142857            |
|The voyager      |70.26666666666667           |234.22222222222223            |
|The nomad        |118.05                      |262.3333333333333             |
|The musicologist |48.483333333333334          |242.41666666666666            |
|The maverick     |159.18333333333334          |238.775                       |
|The jukeboxer    |160.71666666666667          |260.6216216216216             |
|The voyager      |73.85                

In [63]:
result_listeners_pd = result_listeners.toPandas()

In [64]:
result_listeners_pd.to_csv('/content/drive/MyDrive/4th year/Stream Analytics/Spotify_wrap_project/Output csvs/result_listeners.csv', index=False)

In [ ]:
for query in spark.streams.active:
    if query.name == "listening_time_query":
        query.stop()

## Additional Queries

#### Check all records in the df

In [ ]:
query_name = 'df_general_songs'

query = df.writeStream \
    .outputMode("append") \
    .format("memory") \
    .queryName(query_name) \
    .start()


sql_query_df_general = f"""
SELECT *
FROM {query_name}
"""

result_df_general = spark.sql(sql_query_df_general)
result_df_general.show(truncate=False)

+-------+--------+---------+-------+----+--------+-----------+---------+------+-----+----+----------+------------+------+--------------+----------+--------+-----------+
|user_id|latitude|longitude|country|city|timezone|interaction|song_name|artist|genre|year|popularity|danceability|energy|length_seconds|start_time|end_time|personality|
+-------+--------+---------+-------+----+--------+-----------+---------+------+-----+----+----------+------------+------+--------------+----------+--------+-----------+
+-------+--------+---------+-------+----+--------+-----------+---------+------+-----+----+----------+------------+------+--------------+----------+--------+-----------+



In [ ]:
result_df_general.show(truncate=False)

+-------+---------+---------+-------+--------+------------------+-----------------+----------------------------------------------------+-------------------+--------------------+----+----------+------------+------+--------------+-------------+-------------+---------------+
|user_id|latitude |longitude|country|city    |timezone          |interaction      |song_name                                           |artist             |genre               |year|popularity|danceability|energy|length_seconds|start_time   |end_time     |personality    |
+-------+---------+---------+-------+--------+------------------+-----------------+----------------------------------------------------+-------------------+--------------------+----+----------+------------+------+--------------+-------------+-------------+---------------+
|363    |40.49748 |44.7662  |AM     |Hrazdan |Asia/Yerevan      |paused           |I Gotta Feeling                                     |The Black Eyed Peas|dance pop           |2009

In [ ]:
for query in spark.streams.active:
    if query.name == "df_general_songs":
        query.stop()

#### Top 10 songs per user

In [ ]:
selected_df = df.select('user_id', 'song_name', 'interaction')

# Filtering where interaction is "played"
played_df = selected_df.filter(col('interaction') == 'played')

# Aggregate the data by user_id, song_name, and interaction, counting the occurrences
song_stats_df = played_df.groupBy("user_id", "song_name").agg(count("*").alias("occurrences"))

# Define the name for the query
query_name = 'top_songs_per_user'

# Start the streaming query
query = song_stats_df.writeStream \
    .outputMode("update") \
    .format("memory") \
    .queryName(query_name) \
    .start()

# Wait for the stream to finish
#query.awaitTermination()

# SQL to select the top 10 songs per user
sql_query_user_10 = """
SELECT user_id, song_name, occurrences
FROM (
    SELECT user_id, song_name, occurrences,
           ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY occurrences DESC) as rank
    FROM {}
) ranked
WHERE rank <= 10
""".format(query_name)

# Execute the SQL query
result_user_10 = spark.sql(sql_query_user_10)
result_user_10.show()

+-------+---------+-----------+
|user_id|song_name|occurrences|
+-------+---------+-----------+
+-------+---------+-----------+



In [ ]:
result_user_10.show()

+-------+---------+-----------+
|user_id|song_name|occurrences|
+-------+---------+-----------+
+-------+---------+-----------+



In [ ]:
for query in spark.streams.active:
    if query.name == "top_songs_per_user":
        query.stop()